# DataWarehouse Data Transform 2️⃣
- 동일한 컬럼 가지고 있는 데이터끼리 분류하여 '관광지 분류' 컬럼으로 생성  
- 축제행사, 공연 : 날짜 지난 관광지 제거
- 필터링 기능에 포함시킬 컬럼 전처리
  - 요금정보 ['이용요금', '입장료', '상세정보'] : 유료/무료 형태로
  - 유모차 대여 여부 ['유모차 대여 여부'] : 없음/불가/가능
  - 애완동물 동반 가능 여부 ['애완동물 동반 가능 여부'] : 없음/불가/가능
  - 지역 ['지역'] : 경기도/서울/인천

### Packages

In [1]:
# packages
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from datetime import date, datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
from hdfs import InsecureClient
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt #그래프 패키지 모듈 등록
%matplotlib inline 

/tmp/ipykernel_40300/1866876166.py:11: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# matplotlib 한글폰트 
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    rc('font', family='D2Coding')
       
# rc('font', family='NanumGothic') # 나눔폰트 사용시

### DW에 있는 Data 사용하기 위한 Hdfs 클라이언트 객체 생성

In [3]:
client = InsecureClient('http://localhost:9870', user='root')

### Data 불러오기
- travel_data_dropped.json : 불필요한 컬럼을 제거한 raw data 

In [4]:
df_tmp = spark.read.json('/dw_data/travel_data_dropped.json', encoding='utf-8')

In [5]:
df_tmp.printSchema()

root
 |-- 개요: string (nullable = true)
 |-- 개장기간: string (nullable = true)
 |-- 공연시간: string (nullable = true)
 |-- 관람가능연령: string (nullable = true)
 |-- 관람소요시간: string (nullable = true)
 |-- 명칭: string (nullable = true)
 |-- 분류: string (nullable = true)
 |-- 상세정보: string (nullable = true)
 |-- 쉬는날: string (nullable = true)
 |-- 애완동물 동반 가능 여부: string (nullable = true)
 |-- 우편번호: string (nullable = true)
 |-- 유모차 대여 여부: string (nullable = true)
 |-- 이용시간: string (nullable = true)
 |-- 이용시기: string (nullable = true)
 |-- 이용요금: string (nullable = true)
 |-- 입장료: string (nullable = true)
 |-- 전화번호: string (nullable = true)
 |-- 주소: string (nullable = true)
 |-- 주차시설: string (nullable = true)
 |-- 주차요금: string (nullable = true)
 |-- 지역: string (nullable = true)
 |-- 체험가능연령: string (nullable = true)
 |-- 체험안내: string (nullable = true)
 |-- 행사시작일: long (nullable = true)
 |-- 행사종료일: long (nullable = true)



### 동일한 컬럼 가지고 있는 데이터끼리 분류하여 '관광지 분류' 컬럼으로 생성

[관광지 분류]
- 축제공연행사
    - 축제
    - 공연행사 
- 문화시설
    - 문화시설 
- 레포츠
    - 레포츠 
- 인문관광지
    - 휴양관광지
    - 역사관광지
    - 자연관광지
    - 관광자원 

In [11]:
df_tmp.select("분류").distinct().show()

+----------+
|      분류|
+----------+
|휴양관광지|
|    레포츠|
|  문화시설|
|자연관광지|
|역사관광지|
|      축제|
|  공연행사|
|  관광자원|
+----------+



In [13]:
# package
from pyspark.sql import functions as F


# '분류' values <-> '관광지 분류' values 매핑시키기
def map_classification(category):
    if category in ['축제', '공연행사']:
        return '축제공연행사'
    elif category == '문화시설':
        return '문화시설'
    elif category == '레포츠':
        return '레포츠'
    elif category in ['휴양관광지', '역사관광지', '자연관광지', '관광자원']:
        return '인문관광지'
    else:
        return 'Unknown'

mapping_udf = F.udf(map_classification)


# 매핑된 결과 '관광지 분류'로 생성
df_result = df_tmp.withColumn('관광지 분류', mapping_udf(df_tmp['분류']))

In [19]:
# 확인
df_result.filter(df_result['관광지 분류'] == '인문관광지').select('관광지 분류', '분류').distinct().show()
df_result.filter(df_result['관광지 분류'] == '레포츠').select('관광지 분류', '분류').distinct().show()
df_result.filter(df_result['관광지 분류'] == '축제공연행사').select('관광지 분류', '분류').distinct().show()
df_result.filter(df_result['관광지 분류'] == '문화시설').select('관광지 분류', '분류').distinct().show()

+-----------+----------+
|관광지 분류|      분류|
+-----------+----------+
| 인문관광지|휴양관광지|
| 인문관광지|자연관광지|
| 인문관광지|역사관광지|
| 인문관광지|  관광자원|
+-----------+----------+

+-----------+------+
|관광지 분류|  분류|
+-----------+------+
|     레포츠|레포츠|
+-----------+------+

+------------+--------+
| 관광지 분류|    분류|
+------------+--------+
|축제공연행사|공연행사|
|축제공연행사|    축제|
+------------+--------+

+-----------+--------+
|관광지 분류|    분류|
+-----------+--------+
|   문화시설|문화시설|
+-----------+--------+



### '축제공연행사' : 날짜 지난 관광지 제거
- '행사종료일'을 기준으로 

In [21]:
# 모든 rows 보는 세팅
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [23]:
# 날짜 확인
df_result.filter(df_result['관광지 분류'] == '축제공연행사').select('행사시작일', '행사종료일').distinct().show()

+----------+----------+
|행사시작일|행사종료일|
+----------+----------+
|  20231014|  20231020|
|  20230423|  20231112|
|  20231027|  20231028|
|  20230205|  20230205|
|  20231110|  20231111|
|  20230505|  20231112|
|  20231104|  20231105|
|  20230610|  20230617|
|  20231027|  20231027|
|  20231026|  20231126|
|  20231007|  20231029|
|  20230527|  20230730|
|  20230707|  20231014|
|  20240126|  20240218|
|  20230531|  20231203|
|  20231221|  20231231|
|  20230812|  20230815|
|  20230526|  20231226|
|  20231207|  20231207|
|  20230912|  20230913|
+----------+----------+
only showing top 20 rows

